In [68]:
from astropy.io import fits
import numpy as np
import os
from xspec import *

# Number of sources in the table
n_rows = 100

# Generate random values for RA and Dec within realistic ranges
ra_values = np.random.uniform(340, 351, n_rows).astype(np.float32)  # Right Ascension in degrees
dec_values = np.random.uniform(85, 88, n_rows).astype(np.float32)  # Declination in degrees

# Generate random flux values
flux_values = np.random.uniform(1e-12, 1e-11, n_rows).astype(np.float32)

# Choose models from the model_collection
model_names = np.random.choice(os.listdir("model_collection"), n_rows)

# Generate source numbers
source_numbers = np.arange(1, n_rows + 1).astype(np.int32)

# Create FITS columns
col1 = fits.Column(name='SRC_ID', array=source_numbers, format='J')
col2 = fits.Column(name='RA', array=ra_values, format='E', unit='deg')
col3 = fits.Column(name='DEC', array=dec_values, format='E', unit='deg')
col4 = fits.Column(name='IMGROTA', array=np.zeros(n_rows), format='E', unit='deg')
col5 = fits.Column(name='IMGSCAL', array=np.ones(n_rows), format='E', unit='deg')
col6 = fits.Column(name='E_MIN', array=np.ones(n_rows)*0.2, format='E', unit='keV')
col7 = fits.Column(name='E_MAX', array=np.ones(n_rows)*2, format='E', unit='keV')
col8 = fits.Column(name='FLUX', array=flux_values, format='E', unit='erg/s/cm**2')
col9 = fits.Column(name='SPECTRUM', array=model_names, format='100A')  # Temporary placeholder
col10 = fits.Column(name='IMAGE', array=['NULL']*n_rows, format='100A')
col11 = fits.Column(name='TIMING', array=['NULL']*n_rows, format='100A')

# Create Binary Table HDU
cols = fits.ColDefs([col1, col2, col3, col4, col5, col6, col7, col8, col9, col10, col11])
hdu1 = fits.BinTableHDU.from_columns(cols)

# Add required headers
hdu1.header['HDUCLASS'] = 'HEASARC/SIMPUT'
hdu1.header['HDUCLAS1'] = 'SRC_CAT'
hdu1.header['HDUVERS'] = '1.1.0'
hdu1.header['EXTNAME'] = 'SRC_CAT'
hdu1.header['RADESYS'] = 'FK5'
hdu1.header['EQUINOX'] = 2000.0

# Write the first catalog
hdu1.writeto('catalog1.fits', overwrite=True)

# Read the catalog and generate spectral data
pretable = fits.open('catalog1.fits')[1].data

src = []
energies = []
values = []

Xset.chatter = 0

for src_id, modelstring in enumerate(pretable['SPECTRUM']):
    modelstring = modelstring.replace('Model_', '').replace('.xcm', '')
    m = Model(modelstring)
    src.append(str(src_id+1))
    energies.append(m.energies(0)[1:])  # Take the center of the bins
    values.append(m.values(0))

# Create Spectral FITS Table
c1 = fits.Column(name='NAME', array=src, format='100A')
c2 = fits.Column(name='ENERGY', array=energies, format=f'{len(energies[0])}D', unit='keV')
c3 = fits.Column(name='FLUXDENSITY', array=values, format=f'{len(values[0])}D', unit='photon/s/cm**2/keV')

hdu2 = fits.BinTableHDU.from_columns([c1, c2, c3])

# Add required headers for the spectrum table
hdu2.header['HDUCLASS'] = 'HEASARC/SIMPUT'
hdu2.header['HDUCLAS1'] = 'SPECTRUM'
hdu2.header['HDUVERS'] = '1.1.0'
hdu2.header['EXTNAME'] = 'SPECTRUM'

# Write the intermediate FITS file
hdulist = fits.HDUList([fits.PrimaryHDU(), hdu1, hdu2])
hdulist.writeto('final.fits', overwrite=True)

# Replace values in SPECTRUM column (NOT adding a new column)
with fits.open('final.fits', mode='update') as hdul:
    table_data = hdul[1].data  # Get HDU1 data
    spec_column = table_data['SPECTRUM']

    for i in range(len(spec_column)):
        spec_column[i] = f"[SPECTRUM,1][NAME=='{table_data['SRC_ID'][i]}']"

    # Save changes to the file
    hdul.flush()

# Cleanup intermediate files
os.remove('catalog1.fits')

print("Final FITS file 'final.fits' created successfully with updated SPECTRUM column!")



***XSPEC Error:  No valid model component named mymodel
Available model components are:

 Additive Models: 
      agauss       bvcph       cheb6     grbcomp        posm      vmekal
      agnsed     bvequil         cie      grbjet    powerlaw        vnei
     agnslim  bvexpcheb6      compLS        grbm      pshock    vnpshock
        apec     bvgadem      compPS        hatm      qsosed       voigt
       bapec      bvgnei      compST         jet     raymond     vpshock
       bbody       bvnei      compTT      kerrbb       redge    vraymond
    bbodyrad   bvnpshock      compbb       kerrd      refsch       vrnei
     bcempow    bvpshock     compmag    kerrdisk        rnei      vsedov
      bcheb6      bvrnei      comptb     kyrline       sedov      vtapec
        bcie     bvsedov      compth        laor        sirf      vvapec
   bcoolflow     bvtapec    coolflow       laor2      slimbh       vvcie
        bcph     bvvapec         cph      logpar       smaug     vvgadem
      bequil   

Exception: Model Command Error

In [65]:
from astropy import units as u

# Define the values with their units
R = 4 * u.km  # Radius in kilometers
D = 150 * u.pc  # Distance in parsecs
T = 60 * u.eV  # Temperature in electron volts
sigma = 5.670374419e-8 * u.W / (u.m**2 * u.K**4)  # Stefan-Boltzmann constant in W/m^2/K^4

# Convert temperature to Kelvin
T_K = T.to(u.K, equivalencies=u.temperature_energy())

# Apply the formula for flux in SI units (W/m²)
F_SI = sigma * T_K**4 * (R / D)**2

# Convert from W/m² to erg/s/cm²
F_erg_cm2_s = F_SI.to(u.erg / (u.cm**2 * u.s))

F_erg_cm2_s

<Quantity 9.95317308e-12 erg / (s cm2)>

In [67]:
Table.read('final.fits')

SRC_ID,RA,DEC,IMGROTA,IMGSCAL,E_MIN,E_MAX,FLUX,SPECTRUM,IMAGE,TIMING
,deg,deg,deg,deg,keV,keV,erg / (s cm2),,,
int32,float32,float32,float32,float32,float32,float32,float32,bytes100,bytes100,bytes100
1,341.32373,87.09899,0.0,1.0,0.2,2.0,6.5841586e-12,"[SPECTRUM,1][NAME=='1']",NULL,NULL
2,347.8459,87.25366,0.0,1.0,0.2,2.0,8.840939e-12,"[SPECTRUM,1][NAME=='2']",NULL,NULL
3,347.17172,86.78774,0.0,1.0,0.2,2.0,5.224305e-12,"[SPECTRUM,1][NAME=='3']",NULL,NULL
4,350.156,85.444855,0.0,1.0,0.2,2.0,3.3840966e-12,"[SPECTRUM,1][NAME=='4']",NULL,NULL
5,342.6966,87.19449,0.0,1.0,0.2,2.0,4.000203e-12,"[SPECTRUM,1][NAME=='5']",NULL,NULL
6,347.75247,86.9618,0.0,1.0,0.2,2.0,6.4642354e-12,"[SPECTRUM,1][NAME=='6']",NULL,NULL
7,347.77673,85.92146,0.0,1.0,0.2,2.0,2.4395293e-12,"[SPECTRUM,1][NAME=='7']",NULL,NULL
8,349.7555,85.50259,0.0,1.0,0.2,2.0,3.808991e-12,"[SPECTRUM,1][NAME=='8']",NULL,NULL
